In [ ]:
import json
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import re
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from googletrans import Translator
from langdetect import detect, DetectorFactory
import numpy as np
import time
import ast



In [ ]:
df = pd.read_csv("NoDuplicates_Translated2.csv")
df.head()

In [ ]:
print(len(df))

In [ ]:
# parse conversations
text_list = []
text_arrays = []
for conversation in df["vectorized_col_1"]:
    text = ""
    text_arr = []

    # parsed_conversation = ast.literal_eval(conversation)
    conversation = conversation[2:-1]

    values = re.split(r"(?=\{'from': 'human',|\{'from': 'gpt',)", conversation, flags=re.IGNORECASE)
    values = values[1:]
    for value in values:
        
        value = re.sub(r"{'from': 'human', 'value': ", "", value, flags=re.IGNORECASE)
        value = re.sub(r"{'from': 'gpt', 'value': ", "", value, flags=re.IGNORECASE)

        
        text += value
        text_arr.append(value)

    text_list.append(text)
    text_arrays.append(text_arr)

df["text"] = text_list
df["values"] = text_arrays





In [ ]:
df.head()


In [ ]:
df.head()

In [ ]:
corpus = df["text"]

In [ ]:
cached = stopwords.words('english')

def preprocess_text(text):
    text = str(text)
    # remove code
    a=re.compile("<code>.*?</code>")
    text = a.sub('', text)
    # remove angle brackets and all characters inside angle bracket
    a=re.compile("<.*?>")
    text = a.sub('', text)
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # convert to lowercase
    text = text.lower()
    # tokenize the text
    tokens = word_tokenize(text)
    # remove stopwords
    tokens = [word for word in tokens if word not in cached]
    return tokens

# preprocess all text
# preprocessed_text = [' '.join(preprocess_text(document)) for document in corpus]

preprocessed_text = []
i = 0
for document in corpus:
    print(i)
    preprocessed_document = ' '.join(preprocess_text(document))
    preprocessed_text.append(preprocessed_document)
    i += 1





In [ ]:
df["preprocessed_text"] = preprocessed_text

In [ ]:
df.head()

In [ ]:
corpus = df["preprocessed_text"]

In [ ]:
# tokenize the documents
tokenized_documents = [doc.split() for doc in corpus]

# create a dictionary representation of the documents
dictionary = Dictionary(tokenized_documents)

# convert document into bag-of-words format
corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents]


In [ ]:
lda_model = LdaModel(corpus, num_topics=100, id2word=dictionary, random_state=42)


In [ ]:
topics = lda_model.print_topics(num_topics=100)
for topic in topics:
    print(topic)

In [ ]:
# setup: get the model's topics in their native ordering...
all_topics = lda_model.print_topics(num_topics=100)
# ...then create a empty list per topic to collect the docs:
docs_per_topic = [[] for _ in all_topics]

# now, for every doc...
for doc_id, doc_bow in enumerate(corpus):
    # ...get its topics...
    doc_topics = lda_model.get_document_topics(doc_bow)
    # ...& for each of its topics...
    for topic_id, score in doc_topics:
        # ...add the doc_id & its score to the topic's doc list
        docs_per_topic[topic_id].append((doc_id, score))

